In [30]:
import pandas as pd 
import numpy as np
from datetime import timedelta
from datetime import datetime as dt
from datetime import date, timedelta

from functools import partial

In [31]:
def read_process_data(path):
    """
    path: путь к файлу с данными
    return: предобработанные данные
    """
    df = pd.read_excel(path)
    df.columns = ['id', 'type_holiday', 'start', 'end'] # Изменяю названия колонки
    df = df.dropna() # Убираю пропущенные значения
    df = df.drop (index=df.query("id == 'Табельный номер'").index) # Убираю прошлый заголовок
    # Приведение типов данных
    df.start = pd.to_datetime(df.start)
    df.end = pd.to_datetime(df.end)
    df.id = df.id.astype(int)
    return df

In [32]:
def count_weekdays(row):
    """
    row: считанная строка
    """
    start = row['start'].date()
    end = row['end'].date() + timedelta(days=1)
    work_days = np.busday_count(start, end)
    return work_days

In [33]:
def find_necessary_data(path_conditions, df):
    """
    Считать данные из файла  
    Отобрать необходимые значения по типам отпусков
    Сделать срез по датам
    path_conditions: путь до файла с даными о периоде и типов отпусков, 
    которые нужно исключить
    df: предобработанный датафрейм
    """
    with open("conditions.txt", "r", encoding="utf-8") as file:
        lines = [line.strip() for line in file] # итерация по строкам и их запись
    # Поиск необходимых данных из файла conditions.txt
    list_exceptions = lines[lines.index("Исключить:") + 1:]
    date_from = dt.strptime(lines[lines.index("от:") + 1], '%d.%m.%Y')
    date_to = dt.strptime(lines[lines.index("до:") + 1], '%d.%m.%Y')
    # Убираю из файла ненужные строчки по типам отпусков
    df = df[ ~ df.type_holiday.isin (list_exceptions)]
    # Срез по датам
    df = df.query(f"start >= '{date_from}' and start <= '{date_to}'")
    # Проверяю, что конец отпуска не выходит за рамки исследуемого периода
    df.end = df.end.apply(lambda x: x if x <= date_to else date_to)
    # Всего дней отпуска
    df['days'] = df.end - df.start + timedelta(days=1)
    df['work_days'] = df.apply(count_weekdays, axis=1)
    return df

In [56]:
def count_work_days(x):
    """
    Определяет границы отпусков исключая ошибку двойного подсчета
    Считает будние дни 
    x: Series or DataFrame
    return: количество рабочих дней
    """
    x = x.sort_values()
    days = [x.iloc[0]]
    for i in range(1, len(x)):
        flag = 0
        for day in days:
            if day[0] <= x.iloc[i][0] and x.iloc[i][0] <= day[1] and  day[1] <= x.iloc[i][1]:
                day[1] = x.iloc[i][1]
                flag = 1
            elif x.iloc[i][0] <= day[0] and  day[0] <= x.iloc[i][1] and  x.iloc[i][1] <= day[1]:
                day[0] = x.iloc[i][0]
                flag = 1
            elif x.iloc[i][0] >= day[0] and  x.iloc[i][1] <= day[1]:
                flag = 1
        if flag == 0:
            days.append(x.iloc[i])
    work_days = 0
    for day in days:
        work_days += np.busday_count(day[0].date(), day[1].date() + timedelta(days=1))
    return work_days
            

In [57]:
def ad_hoc_task():
    """ 
    Считывает данные из эксель файла, 
    Исключает ненужные данные (по записям из файла conditions.txt)
    Записывает конец отпуска на конец рассматриваемого промежутка, если отпуск заканчивается позже, чем рассматривемый период
    Группирует данные и считает количество будних дней 
    """
    path = 'number_absences_working_days.xlsx'
    path_conditions='conditions.txt'
    df = read_process_data(path)
    df = find_necessary_data(path_conditions, df)
    df['start_end'] = df.apply(date_to_list, axis=1)  
    result = df.groupby('id', as_index=False)\
        .agg({'start_end': count_work_days})\
        .sort_values('start_end', ascending=False)\
        .rename(columns={'id': 'Табельный номер', 'start_end': 'Количество отсуствий'})
    result.to_excel('result.xlsx', index=False)
    print("OK")
    return result, df

In [58]:
result, df = ad_hoc_task()

/var/folders/0m/1wzwyty959g5m50hvj1ddmk80000gp/T/ipykernel_46289/2638881683.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.end = df.end.apply(lambda x: x if x <= date_to else date_to)
/var/folders/0m/1wzwyty959g5m50hvj1ddmk80000gp/T/ipykernel_46289/2638881683.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days'] = df.end - df.start + timedelta(days=1)
/var/folders/0m/1wzwyty959g5m50hvj1ddmk80000gp/T/ipykernel_46289/2638881683.py:24: SettingWithCopyWarning: 
A value is trying to be set on 

OK


In [59]:
result.head()

,Табельный номер,Количество отсуствий
3871,3956,23
2259,2307,20
305,307,18
635,644,18
345,348,18
